In [ ]:
import os
import win32com.client
import numpy as np

###### define groups through which to access all utility functions
VHG_EinP = ['EinpA_bF','EinpA_kF','EinpA_nC','EinpA_nE','EinpF_bF','EinpF_kF','EinpF_nC','EinpF_nE','EinpK_bF','EinpK_kF','EinpK_nC','EinpK_nE','EinpU_bF','EinpU_kF','EinpU_nC','EinpU_nE',]
Mode = ['cbike','ebike']

#Parameter values for C to try out (see Excel 003)
C = [['-0.9'],
     ['-0.4']]
# C[1][1]: 2th row (ebike), 2nd column (1stiteration) = xx

def replace_C(s, new_number):
    parts = s.split('*', 1)
    sign = new_number[0]  # '+' or '-'
    if sign == '+' or sign == '-':
        index = parts[1].find("Matrix(5200)")
        if index != -1 and index > 0:
            parts[1] = parts[1][:index-1] + sign + parts[1][index:]
    return f"{new_number}*{parts[1]}"

distance_bins = [(-1, 0.5), (0.5, 1), (1, 2), (2, 5), (5, 10), (10, 20), (20, 50), (50, 9999)]
distance_bins_agg = [(-1, 1), (1, 2), (2, 5), (5, 10), (10, 20), (20, 9999)]

def print_trip_shares(trip_matrix, dis_matrix, lower_bound, upper_bound, total_trips):
        mask = (dis_matrix > lower_bound) & (dis_matrix <= upper_bound)
        trip_share = round(np.sum(trip_matrix[mask]) / total_trips *100)
        print(f"Anteil Fahrten {lower_bound} - {upper_bound} km: {trip_share}")

def print_trip_shares_bv(trip_matrix, dis_matrix, lower_bound, upper_bound, total_trips):
        distance_mask = (dis_matrix > lower_bound) & (dis_matrix <= upper_bound)
        mask = upper_left_mask & distance_mask
        trip_share = round(np.sum(trip_matrix[mask]) / total_trips *100)
        print(f"Anteil Fahrten {lower_bound} - {upper_bound} km: {trip_share}")

In [ ]:
#Loop over iteration steps to change C, rerun model, compute mode shares by distance classes, and save
for i in range(0,1):
    #verPath = r'D:\ver\20240709_1200.ver'
    verPath = r'C:\Users\arning.FUL\Desktop\ver\DIFF_20250128_0940_C13_0.ver'
    Visum = win32com.client.Dispatch('Visum.Visum.24')
    Visum.loadVersion(verPath)
    visem_einp = Visum.Procedures.Operations.ItemByKey(46)
    
    #change utils for Einpendler
    for mode in Mode:
        for vhg in VHG_EinP:
            
            #get old utilityfunction
            util = visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).AttValue("UtilityFunction")
            
            #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
            if util != "-99999":
                
                #get new C value based on mode / iteration
                if mode == "cbike":
                    new_C = C[0][i]
                elif mode == "ebike":
                     new_C = C[1][i]
                else:
                    print("ERROR")                
            
                #write new utilityfunction
                new_util = replace_C(util,new_C)
                
                #write into Visum
                visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).SetAttValue("UtilityFunction", new_util)

    #change utils for main visem
    visem_main = Visum.Procedures.Operations.ItemByKey(47)
    VHG_main = [
    "BSch_bF", "BSch_kF", "BSch_nC", "BSch_nE",
    "ERWmP_bF", "ERWmP_kF", "ERWmP_nC", "ERWmP_nE",
    "ERWoP_bF", "ERWoP_kF", "ERWoP_nC", "ERWoP_nE",
    "GSch_bF", "GSch_kF", "GSch_nC", "GSch_nE",
    "Kind_bF", "Kind_kF", "Kind_nC", "Kind_nE",
    "NERWmP_bF", "NERWmP_kF", "NERWmP_nC", "NERWmP_nE",
    "NERWoP_bF", "NERWoP_kF", "NERWoP_nC", "NERWoP_nE",
    "Rentner_bF", "Rentner_kF", "Rentner_nC", "Rentner_nE",
    "Sch_bF", "Sch_kF", "Sch_nC", "Sch_nE",
    "Stud_bF", "Stud_kF", "Stud_nC", "Stud_nE"
    ]
    Activity = ["A", "Ax", "B", "E", "F", "Fx", "G", "H", "J", "K", "Kx","S", "U", "Ux", "W", "Y"]
    
    for mode in Mode:
        for vhg in VHG_main:
            for activity in Activity:
                
                #get old utility function
                #not all combinations of vhg and activities exist; skip those exceptions
                try:
                    util = visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).AttValue("UtilityFunction")
                    #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
                    if util != "-99999":

                        #get new C value based on mode / iteration
                        if mode == "cbike":
                            new_C = C[0][i]
                        elif mode == "ebike":
                            new_C = C[1][i]
                        else:
                            print("ERROR")

                        #write new utilityfunction
                        new_util = replace_C(util,new_C)

                        #write into Visum
                        visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).SetAttValue("UtilityFunction", new_util)
                except Exception as e:
                    pass
                    # handle exception (skip the current iteration)
                    # print(f"Skipping vhg: {vhg}, activity: {activity}, mode: {mode} due to error: {e}")
    Visum.Procedures.Execute()

    print("THIS IS FOR ITERATION NUMBER "+str(i))

    # C-Bike
    cbike_trips_matrix = np.array(Visum.Net.Matrices.ItemByKey(51).GetValues())
    cbike_dis_matrix = np.array(Visum.Net.Matrices.ItemByKey(5088).GetValues())
    upper_left_mask = np.zeros_like(cbike_trips_matrix, dtype=bool)
    upper_left_mask[:357, :357] = True
    
    print('C-bike BV')
    Anzcbiketrips = cbike_trips_matrix[upper_left_mask].sum()
    for lower, upper in distance_bins_agg:
        print_trip_shares_bv(cbike_trips_matrix, cbike_dis_matrix, lower, upper, Anzcbiketrips)

    avcbiketriplength = round((cbike_trips_matrix * cbike_dis_matrix * upper_left_mask).sum()/Anzcbiketrips, 1)
    print('C-bike av. trip length: '+ str(avcbiketriplength))
        
    # E-Bike
    ebike_trips_matrix = np.array(Visum.Net.Matrices.ItemByKey(52).GetValues())
    ebike_dis_matrix = np.array(Visum.Net.Matrices.ItemByKey(5089).GetValues())
    upper_left_mask = np.zeros_like(ebike_trips_matrix, dtype=bool)
    upper_left_mask[:357, :357] = True
    
    print('E-bike BV')
    Anzebiketrips = ebike_trips_matrix[upper_left_mask].sum()
    for lower, upper in distance_bins_agg:
        print_trip_shares_bv(ebike_trips_matrix, ebike_dis_matrix, lower, upper, Anzebiketrips)
    
    avebiketriplength = round((ebike_trips_matrix * ebike_dis_matrix * upper_left_mask).sum()/Anzebiketrips, 1)
    print('E-bike av. trip length: '+ str(avebiketriplength))

    verPath_append = r'C:\Users\arning.FUL\Desktop\ver\DIFF_20250128_0940_C13_'+str(i)+'.ver'
    Visum.saveVersion(verPath_append)
    #Visum = None